In [ ]:
# import package
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import PIL.Image as Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  

# Azure環境

In [ ]:
# 使用國網環境，請確定已經下載好Simpson dataset
# 預設資料集就是與此notebook檔案同樣的路徑底下
data_dir = './'
data_path = os.path.join(data_dir, 'simpsons_dataset/')

# 先確定好模型儲存的路徑，預設儲存在與notebook檔案同樣的路徑底下

#要改模型儲存路徑
model_dir = './model-logs/DenseNet'

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# save model
modelfiles = model_dir + '/basic_model-best-model.pth'

# 一、Load Data

## 1. 將圖片路徑以及標籤個存取在陣列裡面
`x_data_list` : 圖片的路徑

`y_data_lsit` : 圖片的標籤

In [ ]:
image_shape = 224

In [ ]:
x_data_list = [] #圖片的路徑
y_data_list = [] #圖片的標籤

# data_path = './simpsons_dataset'
# os.path.join(data_path,dir) = './simpsons_dataset/abraham_grampa_simpson'

for dir in os.listdir(data_path): 
    for img in os.listdir(os.path.join(data_path,dir)): 
        x_data_list.append(os.path.join(data_path,dir,img))
        y_data_list.append(dir)


## 2. List to DataFrame

In [ ]:
data_list = pd.DataFrame({})
data_list['img_path'] = x_data_list
data_list['label'] = y_data_list
data_list


# 二、資料預處理

## 1. Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

train_list, test_list = train_test_split(data_list,
                                         test_size=0.2,
                                         random_state=42,
                                         #按比例分配，如Labela, Labelb的比例是2:1, 則train and test的 Labela, Labelb的比例都會是2:1
                                         stratify=data_list['label'].values,
                                         shuffle = True)
print(len(train_list),len(test_list))


In [ ]:
X_Train = []; y_Train = []
X_Test = []; y_Test = []

for i in range(len(train_list)):
    X_Train.append(train_list.iloc[i].img_path)
    y_Train.append(train_list.iloc[i].label)
    
for i in range(len(test_list)):
    X_Test.append(test_list.iloc[i].img_path)
    y_Test.append(test_list.iloc[i].label)

In [ ]:
print(X_Train[:10])
print(y_Train[:10])

In [ ]:
#隨機查看圖片
import random
def ShowImage(images,labels,total=10): #圖片，標籤，預測標籤，總共顯示張數
    plt.gcf().set_size_inches(18, 20)
    if total >=25:
        total = 25
    for i in range(0,total):
        num = random.randint(0,len(labels)-1)
        img_show = plt.subplot(5, 5, i+1)
        Img = cv2.imread(images[num])
        img_show.imshow(Img[:,:,::-1])
        title = "label = "+str(labels[num])
        plt.title(title)
    plt.show()

ShowImage(X_Train,y_Train,total=25)

## 2. 標籤的Number Encoding

In [ ]:
label_dict = {}

Count = 0
for i in range(len(data_list)):
    if data_list.iloc[i].label in label_dict:
        continue
    else:
        label_dict[data_list.iloc[i].label] = Count
        Count += 1

In [ ]:
labelreverse = {v: k for k, v in label_dict.items()}
labelreverse

## 3. 建立預處理功能以及建立data loader

https://pytorch.org/vision/main/models/generated/torchvision.models.densenet121.html#torchvision.models.densenet121

In [ ]:
# 訓練集的預處理及資料擴增
# resize的default為PIL.Image.BILINEAR
preprocess_train = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.Resize((image_shape,image_shape)),
                                       transforms.RandomAffine(degrees=(-30,30), translate=(0.1, 0.1), scale=(0.9, 1.5), shear=(0,0)),])
                                       #transforms.RandomResizedCrop((image_shape,image_shape))])

# 測試集的預處理
preprocess_test = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                      transforms.Resize((image_shape,image_shape)),])

In [ ]:
def default_loader(path, preprocess_function):
    # 使用cv2讀取圖片
    img_pil = cv2.imread(path)
    # 將BGR 轉換成 RGB
    img_pil = cv2.cvtColor(img_pil, cv2.COLOR_BGR2RGB)
    # 將前面的預處理方式加入圖片中
    img_tensor = preprocess_function(img_pil)
    return img_tensor

In [ ]:
class Custom_Generator(Dataset):
    def __init__(self, image_file, image_label, label_dict, pre_fn, loader = default_loader):
        
        self.images = image_file      # 圖片路徑的陣列
        self.target = image_label     # 圖片標籤的陣列
        self.label_dict = label_dict  # 如：{'dogs': 0, 'cats': 1}
        self.pre_fn = pre_fn          # 資料預處理
        self.loader = loader          # 讀圖片的function

    def __getitem__(self, index):
        fn = self.images[index]      # 圖片路徑
        
        img = self.loader(path = fn, # 圖片讀取以及預處理後的結果，輸入模型用
                          preprocess_function = self.pre_fn)
        
        target = self.label_dict[self.target[index]] # 圖片標籤, 並轉換成數字
        
        return img,target

    def __len__(self):
        return len(self.images)     # 回傳資料集數量

In [ ]:
# 1. 先呼叫我們寫好的Custom_Generator, 將圖片的路徑陣列、圖片的標籤陣列以及預處理功能放入裡面
train_data  = Custom_Generator(image_file = X_Train, 
                               image_label = y_Train, 
                               label_dict = label_dict,
                               pre_fn = preprocess_train)

# 2. 呼叫pytorch的DataLoader來做批次動作，模型的輸入
trainloader = DataLoader(train_data, batch_size=32,shuffle=True)

# 1. 先呼叫我們寫好的Custom_Generator, 將圖片的路徑陣列、圖片的標籤陣列以及預處理功能放入裡面
test_data  = Custom_Generator(image_file = X_Test, 
                              image_label = y_Test, 
                              label_dict = label_dict,
                              pre_fn = preprocess_test)

# 2. 呼叫pytorch的DataLoader來做批次動作，模型的輸入
testloader = DataLoader(test_data, batch_size=32,shuffle=False)



# 三、載入預訓練模型
Use `densenet121` for Transfer learning

## 1. 讀入模型

In [ ]:
import torchvision.models as models

In [ ]:
cnn_model = models.densenet121(pretrained=True)
cnn_model.classifier = nn.Sequential(nn.Linear(in_features = 1024, out_features = 128),  #512 * 7 * 7不能改變
                                     nn.ReLU(True),
                                     nn.Dropout(),
                                     nn.Linear(in_features = 128, out_features = len(labelreverse)),)
cnn_model.to(device)

## 2. 決定要Fine tune Freeze的layer(如果不Freezing可以略過這一步)
這個要判斷今天的資料集狀況再決定是否要對預訓練模型的layer做Freezing

In [ ]:
# 可以先查看所有layers的名稱
for name, param in cnn_model.named_parameters():
    if param.requires_grad:
        print(name)
        
print(cnn_model)

In [ ]:
# # 再決定要layer freeze哪幾層
# for name, param in cnn_model.named_parameters():
#     if param.requires_grad and name == '':
#         break
#     elif param.requires_grad:
#         print(name)
#         param.requires_grad = False
        
cnn_model.to(device)

## 2. 決定 Loss function 、Optimizer

In [ ]:
# 選用Adam為optimizer
# Parameters
# pytorch的CrossEntropyLoss在計算時，會以softmax的輸出作為最後的結果

criterion = nn.CrossEntropyLoss()
lr = 0.0001
epochs = 10
warm_up_epochs = 1

#optimizer_cnn = optim.SGD(cnn_model.parameters(), lr=lr, momentum=0.9)
optimizer_cnn = optim.Adam(cnn_model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_cnn, 
                                                       mode='min', 
                                                       factor=0.1, 
                                                       patience=3, 
                                                       verbose=True)


## 4. 開始訓練囉！

In [ ]:
def PrintLogMessage(Now_epoch, Total_epoch, Now_itr, Total_itr, loss, acc, train = True, flush = False):
    if train == True and flush == True:
        print('[%02d/%02d, %d/%d] loss: %.3f, acc: %.3f' % (Now_epoch, Total_epoch, Now_itr, Total_itr, loss,acc),end = "")
        print("\r", end="", flush=True)
    elif train == True and flush == False:
        print('[%02d/%02d, %d/%d] loss: %.3f, acc: %.3f' % (Now_epoch, Total_epoch, Now_itr, Total_itr, loss,acc),end = "")
    elif train == False:
        print(', test_loss: %.3f, test_acc: %.3f' % (loss,acc))
        

In [ ]:
# 訓練過程
# len(trainLoader) : 訓練集總共的資料量 / batch
# len(trainLoader.dataset) : 訓練集總共的資料量

Best_Acc = 0
cnn_loss_history = []
cnn_acc_history = []

cnn_valloss_history = []
cnn_valacc_history = []

for epoch in range(epochs):
    # 訓練階段
    cnn_model.train()
    running_loss = 0.0
    accuracy = 0.0
    total = 0.0

    for times, data_train in enumerate(trainloader, 0):
        # batch data input
        inputs, labels = data_train
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer_cnn.zero_grad()

        # model Feedforward
        output_train = cnn_model(inputs)
        # Feed forward loss result
        loss = criterion(output_train, labels)
        
        # backward update
        loss.backward()
        
        # optimize
        optimizer_cnn.step()

        # Compute loss
        running_loss += loss.item()
        
        # Compute accuracy
        _, predicted = torch.max(output_train.data, 1)
        total += labels.size(0)
        accuracy += (predicted == labels).sum().item()
        
        # Print log message
        PrintLogMessage(Now_epoch = epoch+1, 
                        Total_epoch = epochs, 
                        Now_itr = times+1, 
                        Total_itr = len(trainloader), 
                        loss = running_loss/(times+1), 
                        acc = accuracy / total, 
                        train = True, 
                        flush = True)
        if times+1 == len(trainloader):
            # Print log message
            PrintLogMessage(Now_epoch = epoch+1, 
                            Total_epoch = epochs, 
                            Now_itr = times+1, 
                            Total_itr = len(trainloader), 
                            loss = running_loss/(times+1), 
                            acc = accuracy / total, 
                            train = True, 
                            flush = False)
            cnn_loss_history.append(running_loss/len(trainloader))
            cnn_acc_history.append(accuracy / total)
            running_loss = 0.0

    # 測試階段
    cnn_model.eval()
    test_loss = 0.0
    accuracy = 0.0
    total = 0.0
    with torch.no_grad(): # disable gradient calculation for efficiency
        for times, data_test in enumerate(testloader, 0):
            # batch data input
            inputs, labels = data_test
            inputs, labels = inputs.to(device), labels.to(device)
            
            # model predict    
            output_test = cnn_model(inputs)
            
            # Compute loss
            loss_t = criterion(output_test, labels)
            test_loss += loss_t.item()
            
            # Compute accuracy
            _, predicted = torch.max(output_test.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
            
            if times+1 == len(testloader):
                scheduler.step(test_loss / len(testloader))
                # Print log message
                PrintLogMessage(Now_epoch = epoch+1, 
                                Total_epoch = epochs, 
                                Now_itr = times+1, 
                                Total_itr = len(testloader), 
                                loss = test_loss/(times+1), 
                                acc = accuracy / total, 
                                train = False, 
                                flush = False)
                if (accuracy / total) > Best_Acc:
                    Best_Acc = (accuracy / total)
                    torch.save(cnn_model, modelfiles)
                    print("Save Model!")
                
                cnn_valloss_history.append(test_loss / len(testloader))
                cnn_valacc_history.append(accuracy / total)
                test_loss = 0.0

print('Finished Training')

# 四、查看模型訓練結果

In [ ]:
def Show_Train_flow(train_cnnmodel, test_cnnmodel, Show = 'loss', Title='Training accuracy comparision'):
    plt.plot(train_cnnmodel)
    plt.plot(test_cnnmodel)
    plt.title(Title)
    plt.ylabel(Show)
    plt.xlabel('Epoch')
    plt.legend(['train','test'])
    plt.show()
    
Show_Train_flow(cnn_loss_history, cnn_valloss_history, Show = 'loss', Title='cnn loss comparision')
Show_Train_flow(cnn_acc_history, cnn_valacc_history, Show = 'acc', Title='cnn acc comparision')

# 五、測試模型
在訓練好模型後，我們一定會拿測試集來測試一下我們訓練模型的好壞

現在就是要使用`kaggle_simpson_testset`來進行預測

## 1. 如果今天想要使用訓練好的模型的話，可以直接load_model


In [ ]:
cnn_model = torch.load(modelfiles)
cnn_model.to(device)
# Set model to eval mode
cnn_model.eval()

## 2. 將預測的資料集讀取下來，並轉換成dataframe形式
** 會全部先設定`class`為1是因為我們不知道標籤，所以先預設為1 **

In [ ]:
#讀取圖片與 Label
testing_dir_path = os.path.join(data_dir, 'kaggle_simpson_testset/predict')
all_file = os.listdir(os.path.join(data_dir, 'kaggle_simpson_testset/predict'))

In [ ]:
# 有原因的
all_file.sort()
test_df = pd.DataFrame({'file_path':all_file,'class':1}) 
test_df

## 3. 將測試集讀入到keras data generator
因為沒有答案，所以務必將`class_mode`設定為`None`

In [ ]:
softmax_output = nn.Softmax(dim=1)

predict_output = []

for i in tqdm(range(len(test_df)),position=0):
    Img = cv2.imread(os.path.join(testing_dir_path,test_df.iloc[i].file_path))
    Img = cv2.cvtColor(Img, cv2.COLOR_BGR2RGB)
    ImgTest = preprocess_test(Img)    # 將圖片轉換成torch tensor(才能丟入模型進行預測)
    ImgTest = torch.unsqueeze(ImgTest, 0)
    ImgTest = ImgTest.to(device)          # to gpu mode
    Result = cnn_model(ImgTest)           # Predict images
    sm_Result = softmax_output(Result)    # softmax output
    
    sm_Result = sm_Result.data.cpu().numpy()
    predict_output.append(sm_Result[0].argmax())

In [ ]:
labelreverse

In [ ]:
y_pred_label = [] # 設定一個將softmax結果轉換成標籤的陣列

for i in range(len(predict_output)):
    y_pred_label.append(labelreverse[predict_output[i]])
    
y_pred_label = np.array(y_pred_label)

In [ ]:
test_df = pd.DataFrame({'file_path':all_file,'pred':y_pred_label})
test_df

# 六、評估模型

## 1. 讀取正確答案

In [ ]:
correct_ans = pd.read_csv(data_dir + "ans.csv")
correct_ans

In [ ]:
correct_ans = correct_ans.sort_values(by=['file'])
correct_ans

## 2. Confusion Matrix

In [ ]:
# 將預測標籤與實際標籤儲存進陣列裡面
y_predict = [];y_TrueLabel=[]

y_TrueLabel = correct_ans['class'].values
y_predict = test_df['pred'].values

In [ ]:
from sklearn.metrics import classification_report

y_TrueLabel = np.asarray(y_TrueLabel)
y_predict = np.asarray(y_predict)
print(classification_report(y_TrueLabel, y_predict, digits = 3))  

In [ ]:
display(pd.crosstab(y_TrueLabel, y_predict, rownames=["True"], colnames=["Predict"]))